In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import math
import random
from ast import literal_eval

2025-04-28 15:18:04.678394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745853484.688745  985715 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745853484.691911  985715 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-28 15:18:04.703462: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class PINN(tf.keras.Model):
    def __init__(self, n_channels, he_initializer = True, floormod=False):
        super().__init__()
        # Fixed metrics
        self.mse_metric = CustomMSE(name='mse')
        self.mae_metric = CustomMAE(name='mae')

        # Use floormod
        self.floormod = floormod

        self.n_channels = n_channels
       
        self.initializer = tf.keras.initializers.HeNormal(1561)
        if not he_initializer:
            self.initializer = tf.keras.initializers.GlorotUniform(1561)
        # input shape of net: (batch size, number of channels, 3 for (x,y,t))
        # output shape of net: (batch size, number of channels, 32)
        # output shape of pressure_net: (batct size, number of channels,)
        # output shape of theta_net: (batct size, number of channels,)
        self.net = tf.keras.Sequential([
            tf.keras.layers.Input(shape=[self.n_channels,3]),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=64, activation='tanh', kernel_initializer = self.initializer)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=64, activation='tanh', kernel_initializer = self.initializer)),
            tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=32, activation='tanh', kernel_initializer = self.initializer))
        ]) 
        
        self.pressure_net = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=32, activation='tanh', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=self.n_channels, activation = 'linear')
        ])
        
        self.theta_net = tf.keras.Sequential([
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=16, activation='softplus', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=8, activation='softplus', kernel_initializer = self.initializer),
            tf.keras.layers.Dense(units=self.n_channels, activation = 'softplus')
        ])
        
        self.lambda_data = tf.keras.Variable(1.0, name='lambda_data', dtype=tf.float32)  
        self.lambda_phys = tf.keras.Variable(-10.0, name='lambda_phys', dtype=tf.float32)  
        self.lambda_theta = tf.keras.Variable(1.0, name='lambda_theta', dtype=tf.float32)  

    def compile(self, *args, **kwargs):
        super().compile(*args, **kwargs)

    def normalize_inputs(self, space_time):
        # Assuming inputs have the shape (batch_size, 3), with (x, y, t)
        # Use precomputed mean and std
        mean = tf.math.reduce_mean(space_time, axis=(0,1))
        std = tf.math.reduce_std(space_time, axis=(0,1))
        return (space_time - mean) / std

    def compute_loss(self, y, y_pred):
        #pres_actual, theta_actual = y_actual[:,0], y_actual[:,1]
        #pres_pred, theta_pred, d2u_dx2, d2u_dy2, d2u_dt2 = y_pred[:,0], y_pred[:,1], y_pred[:,2], y_pred[:,3], y_pred[:,4]
        pres_actual, theta_actual = tf.split(y, 2, axis=-1)
        pres_pred, theta_pred, d2u_dx2, d2u_dy2, d2u_dt2 = tf.split(y_pred, 5, axis=-1)
        
        # data loss
        #data_loss = tf.exp(self.lambda_data)*self.loss.data_loss(pres_actual, pres_pred)
        data_loss = 1.0*self.loss.data_loss(pres_actual, pres_pred)
        
        # physic loss
        #phys_loss= tf.exp(self.lambda_phys)*self.loss.phys_loss(d2u_dx2, d2u_dy2, d2u_dt2)
        phys_loss= 0.001*self.loss.phys_loss(d2u_dx2, d2u_dy2, d2u_dt2)
        
        # theta loss
        #theta_loss = tf.exp(self.lambda_theta)*self.loss.theta_loss(theta_actual, theta_pred)
        theta_loss = 10.0*self.loss.theta_loss(theta_actual, theta_pred)
        
        #tf.print(data_loss, phys_loss, theta_loss)
        return data_loss + phys_loss + theta_loss

    def compute_second_order(self, space_time):
        # Tracking the second order derivatives with respect to space and time
        with tf.GradientTape(persistent=True) as tape2:
            tape2.watch(space_time)
            with tf.GradientTape(persistent=True) as tape1:
                tape1.watch(space_time)
                pres_pred,_ = self(space_time)
                
            # First order derivative: du/dx, du/dy, du/dt
            first_order = tape1.gradient(pres_pred, space_time)

        # Second order derivative: d^2u/dx^2, d^2u/dy^2, d^2u/dt^2
        second_order = tape2.gradient(first_order, space_time)
        
        # free memory
        del tape1, tape2
        return second_order
        
    def train_step(self, dataset):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        # Shape space_time: (batch_size, n_channels, 3)
        # shape pressure: (batch_size, n_channels, 1)
        # Shape theta: (batch_size, 1, 1)
        space_time, pres, theta = dataset
        with tf.GradientTape() as tape:
            # TODO
            pres_pred, theta_pred = self(space_time, training=True)  # Forward pass
            pres_pred = tf.expand_dims(pres_pred, axis=-1)
            theta_pred = tf.expand_dims(theta_pred, axis=-1)

            # Tracking the second order derivatives with respect to space and time
            #d2u_dx2, d2u_dy2, d2u_dt2 = self.compute_second_order(space_time)
            second_order = self.compute_second_order(space_time)

            # Compute the loss value
            # packing for y and y_pred 
            y = tf.concat([pres,theta], axis=-1)
            y_pred = tf.concat([pres_pred, theta_pred, second_order], axis=-1)
            loss = self.compute_loss(y=y, y_pred=y_pred)
        # Compute gradients
        trainable_vars = self.trainable_variables

        gradients = tape.gradient(loss, trainable_vars)
        
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        #self.compiled_metrics.update_state(y, y_pred)
        
        # Update metrics
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(theta, theta_pred)

        return {m.name: m.result() for m in self.metrics}
        
    def test_step(self, dataset):
        space_time, pres, theta = dataset
        pres_pred, theta_pred = self(space_time, training=False)  # Forward pass
        
        pres_pred = tf.expand_dims(pres_pred, axis=-1)
        theta_pred = tf.expand_dims(theta_pred, axis=-1)


        # Tracking the second order derivatives with respect to space and time
        #d2u_dx2, d2u_dy2, d2u_dt2 = self.compute_second_order(space_time)
        second_order = self.compute_second_order(space_time)

        # Compute the loss value
        # packing for y and y_pred 
        y = tf.concat([pres,theta], axis=-1)
        y_pred = tf.concat([pres_pred, theta_pred, second_order], axis=-1)
        #y_pred = tf.concat([pres_pred, theta_pred, d2u_dx2, d2u_dy2, d2u_dt2], axis=1)
        #self.loss.set_lambdas(self.lambda_data, self.lambda_phys, self.lambda_theta)
        loss = self.compute_loss(y=y, y_pred=y_pred)
        
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(theta, theta_pred)
        
        return {m.name: m.result() for m in self.metrics}

    def predict(self, inputs):
        inputs = self.normalize_inputs(inputs)
        output = self.net(inputs)
        pressure = self.pressure_net(output)
        theta = self.theta_net(output)
        if self.floormod:
            # In case the prediction is outside of range of 0 to 360
            theta = tf.math.floormod(theta,360.0)
        return pressure, theta
        
    def call(self, inputs):
        inputs = self.normalize_inputs(inputs)
        output = self.net(inputs)
        pressure = self.pressure_net(output)
        theta = self.theta_net(output)
        return pressure, theta

In [6]:
class PINNLoss(tf.keras.losses.Loss):
    # PINN Loss introducing physic to the model
    # This loss has 3 parts
    # Data loss which is the loss of the pressure field u(x,y,t) , where x,y are space and t is time
    # Physical loss which is the wave equation d^2u/dt^2 = C^2 * (d^2u/dx^2 + d^2u/dy^2)
    # This will help the model understanding the physical constraint and stablizing the model
    # Theta loss is the loss of the theta or angle
    def __init__(self, name="loss"):
        super().__init__(name=name)
        
        # constant sound speed
        # TODO: noisy sound speed
        self.C = 343.0
        
    def theta_loss(self, y_actual, y_pred):
        # cosine loss for loss in the horizontal direction
        # sine loss for loss in the vertical direction
        
        # convert degree to radian
        actual_rad = y_actual*np.pi/180
        pred_rad = y_pred*np.pi/180
    
        # the cosine and sine loss
        cos_loss = tf.math.reduce_mean(tf.math.square(1 - tf.math.cos(actual_rad - pred_rad)))
        sin_loss = tf.math.reduce_mean(tf.math.square(tf.math.sin(actual_rad - pred_rad)))
        
        # loss 
        loss = (cos_loss + sin_loss)
        return loss

    def data_loss(self, pres_actual, pres_pred):
        # Wave pressure at some (x,t)
        loss = tf.math.reduce_mean(tf.math.square(pres_actual - pres_pred))
        return loss
        
    def phys_loss(self, d2u_dx2, d2u_dy2, d2u_dt2):
        # pres_time: the second order derivative of the wave pressure respect to time
        # pres_space: the second order derivative of the wave pressure respect to space
        pres_time = d2u_dt2
        pres_space = (self.C**2) * (d2u_dx2 + d2u_dy2)
        loss = tf.math.reduce_mean(tf.math.square(pres_time - pres_space))
        return loss
        
    def speed_loss(self, c_actual, c_pred):
        # sound speed through medium
        loss = tf.math.reduce_mean(tf.math.square(c_actual - c_pred))
        return loss
        
    def call(self, y_actual, y_pred):
        raise NotImplementedError("Loss is used only as helper, model handles total loss.")

NameError: name 'tf' is not defined

In [5]:
class CustomMSE(tf.keras.metrics.Metric):
    def __init__(self, name='custom_mse', **kwargs):
        super(CustomMSE, self).__init__(name=name, **kwargs)
        # Initialize variables to track squared error and count
        self.total_squared_error = self.add_weight(name='total_squared_error', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Compute squared error with cyclic distance correction
        squared_error = tf.square(((y_true - y_pred) + 180) % 360 - 180)
        
        # Apply sample weight if available
        if sample_weight is not None:
            squared_error = tf.multiply(squared_error, sample_weight)

        # Update the total error and count
        self.total_squared_error.assign_add(tf.reduce_sum(squared_error))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        # Return the mean squared error
        return self.total_squared_error / self.count

    def reset_states(self):
        # Reset the internal variables at the start of each epoch
        self.total_squared_error.assign(0.0)
        self.count.assign(0.0)


NameError: name 'tf' is not defined

In [4]:
class CustomMAE(tf.keras.metrics.Metric):
    def __init__(self, name='custom_mse', **kwargs):
        super(CustomMAE, self).__init__(name=name, **kwargs)
        # Initialize variables to track squared error and count
        self.total_squared_error = self.add_weight(name='total_squared_error', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Compute squared error with cyclic distance correction
        squared_error = tf.abs(((y_true - y_pred) + 180) % 360 - 180)
        
        # Apply sample weight if available
        if sample_weight is not None:
            squared_error = tf.multiply(squared_error, sample_weight)

        # Update the total error and count
        self.total_squared_error.assign_add(tf.reduce_sum(squared_error))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        # Return the mean squared error
        return self.total_squared_error / self.count

    def reset_states(self):
        # Reset the internal variables at the start of each epoch
        self.total_squared_error.assign(0.0)
        self.count.assign(0.0)

NameError: name 'tf' is not defined

In [3]:
def training_evaluation(list_channels, AA_geometry, \
                        inputs, speeds, labels, norm, clambda = [1.0,1.0,1.0],
                        he_initializer=True, floormod=True, 
                        epochs = 50, plot = False, save_fig = False,verbose=False):
    # trained models
    models = []
    # models losses history
    losses = []
    # evluation scores
    evaluates = []
    # reference channel at channel 0 for time delays
    ref_geometry = tf.constant(AA_geometry[0][:2], dtype=float)

    # train models for each selected channels in the list of channels
    for channels in list_channels:
        # Get the geometry of the channels only in x and y planes
        array_geometry = tf.constant(AA_geometry[channels][:,:2], dtype=float)
        # pack selected channels data into tensorflow dataset
        dataset = DataSetPacker(inputs, speeds, labels, channels)
        
        # split dataset into train, validation, and test dataset
        train_dataset, val_dataset, test_dataset = dataset.split(shuffle=False)
        # Initialize the model
        model = MiniPINN(np.shape(channels), array_geometry, he_initializer, floormod)
        # Compile model
        model.compile(optimizer='adam', loss=MiniPINNLoss(array_geometry, ref_geometry, norm, clambda))
        l = model.fit(train_dataset.batch(32), epochs=epochs, validation_data = val_dataset.batch(32), verbose=verbose)
        # Train model
        # Get model score
        eva = EvaluateModel(model, channels, test_dataset)
        eva.evaluate(verbose=True)
        #eva = EvaluateModel.evaluate(model, test_dataset, False)
        if plot:
            if save_fig:
                eva.plot_training(l, save_dir = 'plot/history/')
                eva.plot_evaluation(save_dir = 'plot/evaluation/')
            else:
                eva.plot_training(l)
                eva.plot_evaluation()
        models.append(model)
        losses.append(l)
        evaluates.append(eva)
    return models, evaluates, l
    